
*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  15.01.2024

--- 

# VISUM Model Preprocessing

<!--Notebook description and usage information-->
This notebook processes the raw VISUM traffic model export and converts the data into gpkg files.

In [1]:
import sys

import geopandas as gpd

# import custom modules
sys.path.append('../../utils/')
import data_paths

## Import VISUM traffic model data

The VISUM traffic model provides geoinformation for every road in the region of interest. This includes the indicated speed, the road gradient, road type, hour capacity and the daily average traffic of different vehicle classes.<br>
Additionally, origin-destination matricies, a fundamental input for macroscopic traffic models, are imported. These include the number of starts within the region and will be used to determine cold start emissions.

In [11]:
# import visum data
visum_link_path = data_paths.VISUM_FOLDER_PATH + "analyse_2019_link.SHP"
visum_zone_path = data_paths.VISUM_FOLDER_PATH + "analyse_2019_zone.SHP"

visum_links = gpd.read_file(visum_link_path)
visum_zones = gpd.read_file(visum_zone_path)

print(f"Coordinate reference system of VISUM link data: {visum_links.crs}")
print(f"Coordinate reference system of VISUM zones data: {visum_zones.crs}")

Coordinate reference system of VISUM link data: EPSG:25832
Coordinate reference system of VISUM zones data: EPSG:25832


## Notebook functions

In [12]:
def hbefa_road_type(visum_typeno) -> str: 
    """returns hbefa compatible road type for all road types in the visum traffic model of Munich.
    The information below was retrcollected from the official documentation of the VIUSM model and 
    matched to HBEFA types.
    
    0-10: gesperrte und fiktive Strecken sowie Fuß- und Radwege
    100-199: Autobahnen
    200-299: Bundesstraßen
    300-399: Staatsstraßen
    400-499: Kreisstraßen
    500-599: Gemeindestraßen
    600-699: Hauptstraßen (innerorts)
    700-799: Erschließungsstraßen
    800-899: Mittlerer Ring
    900-999: ÖV Strecken

    Args:
        visum_typeno (string object): defines the road type in the VISUM model

    Returns:
        str: HBEFA compatible road type
    """
    
    int_typeno = int(visum_typeno)
    
    if 0 <= int_typeno <=10: 
        return 'none'
    if 100 <= int_typeno <=199: 
        return 'Motorway-Nat'
    if 200 <= int_typeno <=299: 
        return 'TrunkRoad/Primary-National'
    if 300 <= int_typeno <=399: 
        return 'TrunkRoad/Primary-National'
    if 400 <= int_typeno <=499: 
        return 'Distributor/Secondary'
    if 500 <= int_typeno <=599: 
        return 'Distributor/Secondary'
    if 600 <= int_typeno <=699: 
        return 'Local/Collector'
    if 700 <= int_typeno <=799: 
        return 'Access-residential'
    if 800 <= int_typeno <=899: 
        return 'TrunkRoad/Primary-City'
    if 900 <= int_typeno <=999: 
        return 'none'
    else: 
        return 'none'

In [13]:
def hbefa_road_gradients(visum_gradient) -> str:
    """Converts any road gradient value to the closest gradient 
    value available in HBEFA and returns it as a string.

    Args:
        road_gradient(float): Slope of the road

    Returns:
        str: road gradient string (e.g., '+6%')
    """
    hbefa_gradients = [-6, -4, -2, 0, 2, 4, 6]
    
    hbefa_gradient = min(hbefa_gradients, key=lambda x: abs(x - visum_gradient))
    hbefa_gradient_string = str(hbefa_gradient)+'%'
    
    return hbefa_gradient_string

In [14]:
def hbefa_speed_value(hbefa_road_type:str,
                     speed:int) -> int:
    
        """Converts speed value to closest speed value available in HEBFA

        Args:
            road_type (str): road type 
            speed (int): speed value

        Returns:
            int: Speed value available in HBEFA
        """
        try:
            hbefa_speed = {'Motorway-Nat': [80, 90, 100, 110, 120, 130],
                        'Motorway-City': [60, 70, 80, 90, 100, 110],
                        'TrunkRoad/Primary-National': [70, 80, 90, 100, 110, 120],
                        'TrunkRoad/Primary-City': [50, 60, 70, 80, 90],
                        'Distributor/Secondary': [30, 40, 50, 60, 70, 80],
                        'Local/Collector': [30, 40, 50, 60],
                        'Access-residential': [30, 40, 50]}
            
            hbefa_speed = min(hbefa_speed[hbefa_road_type], key=lambda x: abs(x - speed))
        except KeyError as e: 
            print('error ' + str(e))
            print(hbefa_road_type)
            print(speed)
        return hbefa_speed

## Data processing
- Drop columns with DTV < 1
- Convert road type, speed and gradient to HBEFA compatible values
- Drop columns with undefined road type
- Rename columns

In [15]:
# drop columns where the daily traffic ('SUMME_KFZ') is < 1 vehicle
visum_links = visum_links.drop(visum_links.loc[(visum_links['SUMME_KFZ'] < 1)].index, axis = 0)

# add road type to visum link data and drop entries with 'None' road type
visum_links['road_type'] = visum_links.apply(lambda row: hbefa_road_type(row['TYPENO']), axis = 1)
visum_links = visum_links.drop(visum_links.loc[(visum_links['road_type'] == 'none')].index, axis = 0)

# convert road gradient values
visum_links['hbefa_gradient'] = visum_links.apply(lambda row: hbefa_road_gradients(row['SLOPE']), axis = 1)

# convert hbefa speed values
visum_links['hbefa_speed'] = visum_links.apply(lambda row: hbefa_speed_value(speed = row['V0PRT'],
                                                                             hbefa_road_type = row['road_type']), axis =1)

# aggregate heavy vehicle rows
visum_links['dtv_HGV'] = visum_links[['BEL_LKW_M','BEL_LKW_G']].sum(axis = 1)

# rename rows
visum_links.rename(columns={
    'SUMME_KFZ':'dtv_SUM',
    'NO':'road_link_id',
    'BEL_PKW':'dtv_PC',
    'BEL_LKW_K':'dtv_LCV',
    'CAPPRT':'hour_capacity',
    'V0PRT':'speed'
}, inplace = True)

# reset index
visum_links = visum_links.reset_index(drop = True)

In [16]:
# calculate vehicle shares of the vehicle classes
visum_links['delta_HGV'] = visum_links['dtv_HGV'] / visum_links['dtv_SUM']
visum_links['delta_LCV'] = visum_links['dtv_LCV'] / visum_links['dtv_SUM']
visum_links['delta_PC'] = visum_links['dtv_PC'] / visum_links['dtv_SUM']

## Distribute cold starts

In [17]:
# add empty columns to the visum links file for cold starts
visum_zones.rename(columns={
    'qv_pkw':'PC_starts',
    'qv_lkw_k':'LCV_starts',
}, inplace = True)

visum_links[['PC_cold_starts', 'LCV_cold_starts']] = 0.0

# evenly distribute the number of starts to all road links within a buffered region around the origin of starts
buffer_distance = 2000

for idx, row in visum_zones.iterrows():
    geometry = row.geometry.buffer(buffer_distance)
    geometry = gpd.GeoDataFrame(geometry=[row.geometry.buffer(buffer_distance)], crs=25832)
    combined = gpd.sjoin(geometry, visum_links, how="right", predicate='intersects').dropna()
    
    sum_dtv = combined['dtv_SUM'].sum()
    visum_links.loc[combined.index, 'PC_cold_starts']  += visum_links.loc[combined.index, 'dtv_SUM'] * row['PC_starts']/sum_dtv
    visum_links.loc[combined.index, 'LCV_cold_starts'] += visum_links.loc[combined.index, 'dtv_SUM'] * row['LCV_starts']/sum_dtv

## Save processed data

In [18]:
links_columns_to_keep = ['road_link_id', 'road_type', 'hour_capacity', 'hbefa_speed', 
                         'hbefa_gradient', 'dtv_SUM', 'delta_PC', 'delta_LCV', 'delta_HGV',
                         'PC_cold_starts', 'LCV_cold_starts', 'geometry']

visum_links[links_columns_to_keep].to_file(data_paths.VISUM_FOLDER_PATH+'visum_links.gpkg', driver='GPKG')